<a href="https://colab.research.google.com/github/ananthakrishnanrr/pyspark/blob/main/pyspark_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:
df1=spark.read.format("csv").option("header","true").load("original.csv")

In [4]:
df1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [5]:
from pyspark.sql.functions import *

In [6]:
df2=df1.withColumn("clean_city",when(df1.City.isNull(), 'Unknown').otherwise(df1.City))

In [7]:
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [8]:
df2=df2.filter(df2.JobTitle.isNotNull())

In [9]:
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
|  8|   Goddart|     Flear|  Mal

In [10]:
df2=df2.withColumn("clean_salary",df2.Salary.substr(2,100).cast("float"))

In [11]:
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [12]:
avg_salary=df2.groupBy().avg("clean_salary").take(1)[0][0]

In [13]:
print(avg_salary)

55516.32088199837


In [14]:
df2=df2.withColumn("new_salary",when(df2.clean_salary.isNull(),lit(avg_salary)).otherwise(df2.clean_salary))

In [15]:
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 3

In [23]:
import numpy as np

In [26]:
Lati=df2.select("Latitude")

In [27]:
Lati.show()

+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|      null|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
+----------+
only showing top 20 rows



In [28]:
Lati=Lati.filter(Lati.Latitude.isNotNull())

In [29]:
Lati.show()

+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
|48.4902808|
+----------+
only showing top 20 rows



In [30]:
Lati=Lati.withColumn("Latitude2",Lati.Latitude.cast("float")).select("Latitude2")

In [31]:
Lati.show()

+----------+
| Latitude2|
+----------+
| 50.577408|
|  48.82316|
| 44.504723|
| 53.426613|
| 45.190517|
| 32.027935|
|  4.272793|
|     -5.85|
|  39.17238|
|  49.81518|
|  42.10148|
|  49.79233|
| 43.494576|
| 52.744167|
| 38.696247|
|-7.7232566|
| 40.717205|
|  49.16291|
| 40.757683|
|  48.49028|
+----------+
only showing top 20 rows



In [32]:
median=np.mean(Lati.collect())

In [33]:
print(median)

25.417463958980804


In [34]:
df2=df2.withColumn("Lat",when(df2.Latitude.isNull(),lit(median)).otherwise(df2.Latitude))

In [35]:
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|               Lat|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|        50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|        48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|

In [44]:
gender_sal=df2.groupBy("gender").agg(avg("new_salary").alias("avg_salary"))

In [45]:
gender_sal.show()

+------+------------------+
|gender|        avg_salary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [52]:
df2=df2.withColumn("Female_salary",when(df2.gender == 'Female', df2.new_salary).otherwise(lit(0)))
df2=df2.withColumn("Male_salary",when(df2.gender == 'Male', df2.new_salary).otherwise(lit(0)))


In [53]:
df2.show(10)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+----------------+--------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|               Lat|   Female_salary|   Male_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+----------------+--------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|        50.5774075|   57438.1796875|           0.0|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|        48.8231572|   62846.6015625|      

In [56]:
jobtitle_sal=df2.groupBy("JobTitle").agg(avg("Female_salary").alias("avg_Female_salary"),avg("Male_salary").alias("avg_Male_salary"))

In [57]:
jobtitle_sal.show()

+--------------------+------------------+------------------+
|            JobTitle| avg_Female_salary|   avg_Male_salary|
+--------------------+------------------+------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|
|   Media Manager III|29586.436197916668|17381.920572916668|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|
|       Geologist III|      31749.046875|    12830.75390625|
|        Geologist II|               0.0|   43293.865234375|
|Database Administ...|               0.0|     52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625|
|Software Engineer II|               0.0|      74782.640625|
|       Accountant IV|   82732.248046875|               0.0|
|    Product Engineer|    41825.48359375|       20464.94375|
|Software Test Eng...|  32218.6083984375|   27122.462890625|
|Safety Technician...|               0.0|   29421.529296875|
|    Junior Executive|15

In [58]:
jobtitle_sal=jobtitle_sal.withColumn("Delta",jobtitle_sal.avg_Female_salary - jobtitle_sal.avg_Male_salary)

In [59]:
jobtitle_sal.show()

+--------------------+------------------+------------------+-------------------+
|            JobTitle| avg_Female_salary|   avg_Male_salary|              Delta|
+--------------------+------------------+------------------+-------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|   35049.5244140625|
|   Media Manager III|29586.436197916668|17381.920572916668|       12204.515625|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|  8464.957356770836|
|       Geologist III|      31749.046875|    12830.75390625|     18918.29296875|
|        Geologist II|               0.0|   43293.865234375|   -43293.865234375|
|Database Administ...|               0.0|     52018.4609375|     -52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|   -16252.279296875|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625| -4636.834765625001|
|Software Engineer II|               0.0|      74782.640625|      -74782.640625|
|       Accountant IV|   827

In [60]:
df2.show(10)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+----------------+--------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|               Lat|   Female_salary|   Male_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+----------------+--------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|        50.5774075|   57438.1796875|           0.0|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|        48.8231572|   62846.6015625|      

In [61]:
city_avg=df2.groupBy("City").agg(avg("new_salary").alias("avg_city"))

In [70]:
city_avg.show(10)

+----------------+----------------+
|            City|        avg_city|
+----------------+----------------+
|       Sułkowice|  33432.98828125|
|         Klippan|     77039.46875|
|     Trollhättan|53311.6845703125|
|       Shinaihai|    39544.640625|
|        Hongzhou|  35707.30859375|
|        Cipinang| 11617.509765625|
|Viejo Daan Banua|         43927.5|
|        Tsiatsan| 18795.439453125|
|      San Andres|  52426.80078125|
|          Krasna|   72022.7890625|
+----------------+----------------+
only showing top 10 rows



In [71]:
 city_avg=city_avg.sort(col('avg_city').desc())

In [72]:
city_avg.show()

+-----------------+-------------+
|             City|     avg_city|
+-----------------+-------------+
|        Mesopotam|  99948.28125|
|       Zhongcheng| 99942.921875|
|           Caxias|99786.3984375|
|      Karangtawar|99638.9921875|
|        Itabaiana|  99502.15625|
|           Pasian|  99421.34375|
|           Webuye| 99368.546875|
|      Yuktae-dong| 99250.828125|
|           Zinder|  99222.84375|
|   Timiryazevskiy|   99142.9375|
|        Sawahbaru|99013.7109375|
|          Madimba|98737.8671875|
|         Huangshi|  98690.34375|
|          Gharyan|   98679.3125|
|         Yŏnan-ŭp| 98628.609375|
|     Wringinputih|98603.8203125|
|Monte da Boavista|  98586.71875|
|          Klukeng|98439.4921875|
|         Murmashi|  98226.15625|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows

